#                                    The Battle of Neighborhoods

Our client wants to open a new Univerity in the UAE but is confused between the differrent Emirates. He is looking forward to choose the Emirate that has the lowest number of universities. 
Let's help our client by suggesting the best Emirate to open a new university. We will use Foursqaure to compare the number of univerities in 3 Emirates i.e. Dubai, Sharjah and Abu Dhabi.

### Importing/Installing the Libraries 

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Foursquare Credentials

In [2]:
CLIENT_ID = 'P1JBH0T4YESTJE1CI4QDDUI5H3NDIGIYBRLJDP2CLAWMUWUJ' # your Foursquare ID
CLIENT_SECRET = 'RZXXSDZHDGJHLRRIW1WMS1WASLSSS0U2PMBUDWHHI5NZ43U5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: P1JBH0T4YESTJE1CI4QDDUI5H3NDIGIYBRLJDP2CLAWMUWUJ


### Defining the Emirates of UAE 

In [36]:
LIMIT = 500 
cities = ['Dubai, DXB', 'Abu Dhabi, AUH', 'Sharjah, SHJ']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ae941735") # University CATEGORY ID
    results[city] = requests.get(url).json()

In [37]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']


/Users/afiqhassan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


## Initial Visualizations 

Let's visualize the initial results on map to get a general idea.

In [44]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Universities in {city} = ", results[city]['response']['totalResults'])


Total number of Universities in Dubai, DXB =  66
Total number of Universities in Abu Dhabi, AUH =  16
Total number of Universities in Sharjah, SHJ =  39


In [46]:
maps[cities[0]]

In [48]:
maps[cities[1]]

In [47]:
maps[cities[2]]

### Checking the Distance from Mean Coordinates

The maps above are giving us a general idea about the number of existing universities in each Emirate. However we are yet to know the distance of the universities from the center of each city. Therefore let's find out the distnace od universities from mean coordinates.

In [50]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Dubai, DXB
Distance from Mean coordinates
0.11670375465046973
Abu Dhabi, AUH
Distance from Mean coordinates
0.07813806472677444
Sharjah, SHJ
Distance from Mean coordinates
0.06875580201947419


In [41]:
maps[cities[0]]

In [42]:
maps[cities[1]]

In [43]:
maps[cities[2]]

From the above maps we can visualize that the Emirates of Dubai and Sharjah have the most number of universities and that too very near from the Mean Coordinates. The least number of universities are present in Abu Dhabi. Recall tha our client was interested in opening a university in the Emirate that has the lowest number of existing univerities.


### Thus, based on the data obtained from Fourquare and after visualizing the above maps, we can conclude that the Emirate of Abu Dhabi would be the best to open a new university.